In [13]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [17]:
# Define paths and parameters
train_data_path = 'C:\\Users\\aksha\\OneDrive\\Desktop\\AgriMitra\\datasets\\crop_disease\\PlantVillage'  # Update this path
input_shape = (224, 224, 3)
batch_size = 32
epochs = 10

In [18]:
# Data augmentation and preparation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [19]:
# Create data generator
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 16221 images belonging to 15 classes.


In [20]:
# Number of classes
num_classes = train_generator.num_classes

# Load pre-trained ResNet50 model + higher level layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

In [21]:
# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Correct number of classes

In [22]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

In [23]:
# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

In [24]:
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
# Train the model
model.fit(train_generator, epochs=epochs)

Epoch 1/10


c:\Users\aksha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


507/507 ━━━━━━━━━━━━━━━━━━━━ 2542s 5s/step - accuracy: 0.1779 - loss: 2.4180
Epoch 2/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 14441s 29s/step - accuracy: 0.2709 - loss: 2.1472
Epoch 3/10
  4/507 ━━━━━━━━━━━━━━━━━━━━ 2:02:41 15s/step - accuracy: 0.2832 - loss: 2.1176

In [ ]:
# Optionally unfreeze some layers and fine-tune
for layer in base_model.layers[-10:]:
    layer.trainable = True

In [ ]:
model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=epochs)